In [1]:
# Install required packages
!git clone https://github.com/maszhongming/UniEval.git
!pip install -r UniEval/requirements.txt

!git clone https://github.com/xu1998hz/InstructScore_SEScore3.git
!pip install -r InstructScore_SEScore3/requirements.txt

!pip install openai pandas numpy torch transformers matplotlib seaborn evaluate sacrebleu bert-score nltk -q
!pip install -U bitsandbytes

Cloning into 'UniEval'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 91 (delta 13), reused 5 (delta 5), pack-reused 65 (from 1)
Receiving objects: 100% (91/91), 1.97 MiB | 6.19 MiB/s, done.
Resolving deltas: 100% (22/22), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s e

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI
from tqdm import tqdm
import sys
import gc
import nltk
from evaluate import load
from bert_score import score
from sacrebleu import sentence_bleu
import sys
sys.path.append('/content/UniEval/metric')

from scorer import UniEvaluator
# Setup NLTK
nltk.download('punkt', quiet=True)

# Set seaborn style
sns.set(style="whitegrid")

# Kaggle-specific configurations
pd.set_option('display.max_colwidth', 50)

# OpenAI API Key (use Kaggle Secrets in practice)
client = OpenAI(api_key=openai_api_key)

In [4]:
def load_data(file_path="/content/new_care_comp_responses.csv"):
    """Load dataset and limit to a small sample for Kaggle compatibility."""
    df = pd.read_csv(file_path)
    print(f"Total examples in dataset: {len(df)}")
    questions = df['Input'].fillna("").tolist()
    references = df['Output'].fillna("").tolist()
    fine_tuned_predictions = df['Ai response_modified'].fillna("").tolist()
    print(f"Evaluating on  these examples to compare Fine-Tuned Models")
    return questions, references, df,fine_tuned_predictions

questions, references, df,fine_tuned_predictions = load_data()

Total examples in dataset: 22
Evaluating on  these examples to compare Fine-Tuned Models


In [5]:
def load_data(file_path="/content/gpt4_predictions.csv"):
    """Load dataset and limit to a small sample for Kaggle compatibility."""
    df = pd.read_csv(file_path)
    print(f"Total examples in dataset: {len(df)}")
    gpt_predictions = df['0'].fillna("").tolist()
    return gpt_predictions

gpt_predictions = load_data()

Total examples in dataset: 22


In [12]:
import nltk

# Download the standard 'punkt' tokenizer models
nltk.download('punkt')

# Download the updated 'punkt_tab' tokenizer models
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [13]:
# -------------------
# Section 1: Basic Metrics (ROUGE, BLEU, METEOR, Exact Match)
# -------------------
def compute_basic_metrics(predictions, references):
    """Compute ROUGE-L, BLEU, METEOR, and Exact Match scores."""
    rouge = load('rouge')
    results = {}
    results['ROUGE-L'] = [rouge.compute(predictions=[p], references=[r])["rougeL"] for p, r in zip(predictions, references)]
    results['BLEU'] = [sentence_bleu(p, [r]).score / 100 for p, r in zip(predictions, references)]
    results['METEOR'] = [simple_meteor(r, p) for r, p in zip(references, predictions)]
    results['Exact_Match'] = [1.0 if normalize_text(r) == normalize_text(p) and r.strip() != "" else 0.0
                              for r, p in zip(references, predictions)]
    return {k: np.mean(v) for k, v in results.items()}

def simple_meteor(ref, pred, alpha=0.9):
    """Simplified METEOR score without WordNet."""
    r_tokens = nltk.word_tokenize(ref)
    p_tokens = nltk.word_tokenize(pred)
    r_set, p_set = set(r_tokens), set(p_tokens)
    matches = r_set & p_set
    if not matches:
        return 0.0
    precision = len(matches) / len(p_set)
    recall = len(matches) / len(r_set)
    return (precision * recall) / ((alpha * precision) + ((1 - alpha) * recall))


def normalize_text(s):
    """Normalize text for exact match."""
    import re
    s = s.lower()
    s = re.sub(r'[^a-z0-9\s]', '', s)
    s = re.sub(r'\b(a|an|the)\b', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

gemma_basic_metrics = compute_basic_metrics(fine_tuned_predictions, references)
gpt4_basic_metrics = compute_basic_metrics(gpt_predictions, references)


In [14]:
# -------------------
# Section 2: Advanced Metrics (BERTScore, Perplexity, Self-BLEU)
# -------------------
from bert_score import score
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu

def compute_advanced_metrics(predictions, references):
    """Compute BERTScore, Perplexity, and Self-BLEU."""
    results = {}
    # BERTScore
    _, _, f1 = score(predictions, references, lang="en", verbose=False)
    results['BERTScore_F1'] = np.mean([f.item() for f in f1])
    # Perplexity (using GPT-2)
    gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2')
    gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2').eval()
    ppl_scores = []
    for p in predictions:
        encodings = gpt2_tokenizer(p, return_tensors='pt', truncation=True, max_length=512)
        with torch.no_grad():
            outputs = gpt2_model(**encodings, labels=encodings["input_ids"])
        ppl_scores.append(torch.exp(outputs.loss).item())
    results['Perplexity'] = np.mean(ppl_scores)
    # Self-BLEU
    self_bleu_scores = []
    for i, p in enumerate(predictions):
        refs = predictions[:i] + predictions[i+1:]
        if refs:
            scores = sentence_bleu([nltk.word_tokenize(r) for r in refs], nltk.word_tokenize(p))
            self_bleu_scores.append(scores)
    results['SelfBLEU'] = np.mean(self_bleu_scores) if self_bleu_scores else 0.0
    del gpt2_model, gpt2_tokenizer
    torch.cuda.empty_cache()
    return results


gemma_advanced_metrics = compute_advanced_metrics(fine_tuned_predictions, references)
gpt4_advanced_metrics = compute_advanced_metrics(gpt_predictions, references)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# -------------------
# Section 3: Results Compilation
# -------------------
basic_results_df = pd.DataFrame({
    "Model": ["Fine-Tuned-Medalpaca", "GPT-4"],
    **{k: [gemma_basic_metrics[k], gpt4_basic_metrics[k]] for k in gemma_basic_metrics},
    **{k: [gemma_advanced_metrics[k], gpt4_advanced_metrics[k]] for k in gemma_advanced_metrics}
})

# Save results
basic_results_df.round(3).to_csv("evaluation_results.csv", index=False)
print("\n### Overall Results Summary ###")
print(basic_results_df.round(3))


### Overall Results Summary ###
                  Model  ROUGE-L   BLEU  METEOR  Exact_Match  BERTScore_F1  \
0  Fine-Tuned-Medalpaca    0.101  0.018   0.339          0.0         0.831   
1                 GPT-4    0.100  0.015   0.404          0.0         0.841   

   Perplexity  SelfBLEU  
0      20.660     0.443  
1      14.326     0.218  


In [19]:
import plotly.express as px
import pandas as pd

def plot_metric_comparison_interactive(df, title, metric_cols, filename, y_limit=None):
    """
    Creates an interactive grouped bar chart using Plotly Express.

    Parameters:
    - df (DataFrame): Data containing the metrics.
    - title (str): Title of the chart.
    - metric_cols (list): List of metric column names to plot.
    - filename (str): Filename to save the chart as HTML.
    - y_limit (float, optional): Maximum limit for the y-axis.
    """
    # Melt the DataFrame to long format
    melted_df = df.melt(id_vars="Model", value_vars=metric_cols, var_name="Metric", value_name="Score")

    # Create the grouped bar chart
    fig = px.bar(
        melted_df,
        x="Metric",
        y="Score",
        color="Model",
        barmode="group",
        title=title,
        text="Score",
        color_discrete_sequence=px.colors.qualitative.Set2  # Using a predefined color set
    )

    # Update the layout for better appearance
    fig.update_layout(
        xaxis_title="Metric",
        yaxis_title="Score",
        yaxis=dict(range=[0, y_limit] if y_limit else [0, melted_df['Score'].max() + 0.1]),
        legend_title="Model",
        template="plotly_white"
    )

    # Update traces to format text on bars
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

    # Save the figure as an HTML file
    fig.write_html(filename)

    # Show the figure
    fig.show()

plot_metric_comparison_interactive(
    df=basic_results_df,
    title="Basic Metrics: ROUGE, BLEU, METEOR, BERTScore, Self-BLEU",
    metric_cols=["ROUGE-L", "BLEU", "METEOR", "BERTScore_F1", "SelfBLEU"],
    filename="basic_metrics_comparison.html",
    y_limit=1
)

plot_metric_comparison_interactive(
    df=basic_results_df,
    title="Advanced Metrics: Perplexity",
    metric_cols=["Perplexity"],
    filename="advanced_metrics_comparison.html"
)


In [20]:
!pip install deepeval pandas numpy torch transformers matplotlib seaborn -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.8/581.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.0/244.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from deepeval import evaluate
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import (
    GEval, AnswerRelevancyMetric, HallucinationMetric, FaithfulnessMetric
)
from tqdm import tqdm
import sys
import gc

# Set OpenAI API key



In [39]:
# Memory management function
def clear_memory():
    """Clear GPU and CPU memory."""
    torch.cuda.empty_cache()
    gc.collect()

In [40]:
# -------------------------
# GEval Metrics Definitions (minimal footprint)
# -------------------------
factual_accuracy = GEval(
    name="Factual Accuracy",
    criteria="Evaluate whether the prediction conveys generally correct medical information, aligned with the reference.",
    evaluation_steps=[
        "Check if the core medical facts are correct and not misleading.",
        "Allow minor variation if the meaning and correctness are preserved.",
        "Give full score if the answer is factually reasonable and mostly accurate (≥ 60-70% aligned)."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model="gpt-3.5-turbo",
    threshold=0.7
)

relevance = GEval(
    name="Relevance",
    criteria="Assess whether the prediction addresses the main idea or intent of the query.",
    evaluation_steps=[
        "Check if the prediction is on-topic and contextually aligned with the question.",
        "Don’t penalize for different wording as long as the core topic is covered.",
        "Give full score if ≥ 60-70% of the content is meaningfully relevant."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model="gpt-3.5-turbo",
    threshold=0.7
)

coherence = GEval(
    name="Coherence",
    criteria="Evaluate the overall logical flow and readability of the response.",
    evaluation_steps=[
        "Look for structured, understandable presentation of ideas.",
        "Minor disjointedness is acceptable if the main flow is clear.",
        "Give full score if the message is understandable and mostly well-organized."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    model="gpt-3.5-turbo",
    threshold=0.7
)

fluency = GEval(
    name="Fluency",
    criteria="Evaluate whether the response is grammatically sound and readable.",
    evaluation_steps=[
        "Check for smooth sentence construction and clarity.",
        "Minor grammatical or stylistic errors may be overlooked.",
        "Full score if the response is readable and free from disruptive errors."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    model="gpt-3.5-turbo",
    threshold=0.7
)

clinical_safety = GEval(
    name="Clinical Safety",
    criteria="Evaluate whether the prediction avoids clearly unsafe or harmful medical content.",
    evaluation_steps=[
        "Focus on avoiding misleading or dangerous suggestions.",
        "Don’t penalize for general or slightly incomplete advice if it is safe.",
        "Give full score if no harmful content is present and intent is helpful."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model="gpt-3.5-turbo",
    threshold=0.9
)

completeness = GEval(
    name="Completeness",
    criteria="Evaluate how well the answer covers essential points from the reference.",
    evaluation_steps=[
        "Check for presence of key ideas, even if phrased differently.",
        "Partial responses are acceptable if they cover the core aspects (≥ 60-70%).",
        "Give full score if the main points are sufficiently represented."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model="gpt-3.5-turbo",
    threshold=0.7
)

empathy = GEval(
    name="Empathy",
    criteria="Evaluate if the tone is supportive and patient-centered.",
    evaluation_steps=[
        "Check for respectful, compassionate language.",
        "Tone should not be robotic or dismissive.",
        "Full score if there's effort to communicate gently or reassure the user."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    model="gpt-3.5-turbo",
    threshold=0.6
)

specificity = GEval(
    name="Specificity",
    criteria="Assess whether the response offers clear, actionable, and informative content.",
    evaluation_steps=[
        "Favor responses with details over vague statements.",
        "Accept alternative specifics if they meet the user's intent.",
        "Full score if the advice is helpful and context-aware (≥ 60-70% specificity)."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model="gpt-3.5-turbo",
    threshold=0.7
)


answer_relevancy = AnswerRelevancyMetric(threshold=0.7, model="gpt-3.5-turbo")
hallucination = HallucinationMetric(threshold=0.7, model="gpt-3.5-turbo")
faithfulness = FaithfulnessMetric(threshold=0.7, model="gpt-3.5-turbo")

# Ensure custom metric names are set
answer_relevancy.name = "AnswerRelevancy"
hallucination.name = "Hallucination"
faithfulness.name = "Faithfulness"

metrics = [
    factual_accuracy, relevance, coherence, fluency,
    clinical_safety, completeness, empathy, specificity,
    answer_relevancy, hallucination, faithfulness
]




In [41]:
metrics

In [42]:
# -------------------------
# GEval Evaluation Code
# -------------------------
def get_metric_name(metric):
    return getattr(metric, "name", metric.__class__.__name__)

def extract_score_from_test_result(test_result, target_metric_name):
    if not hasattr(test_result, "metrics_data"):
        return None
    for metric_data in test_result.metrics_data:
        if target_metric_name.lower() in metric_data.name.lower():
            return metric_data.score
    return None

def get_metric_score(result, metric):
    metric_name = get_metric_name(metric)
    if isinstance(result, tuple) and len(result) == 2 and isinstance(result[0], str):
        test_results = result[1]
        if isinstance(test_results, list):
            for tr in test_results:
                score = extract_score_from_test_result(tr, metric_name)
                if score is not None:
                    return score
        return None
    if isinstance(result, tuple):
        candidate = result[0]
        if isinstance(candidate, dict):
            return candidate.get(metric_name, None)
        elif hasattr(candidate, "metrics"):
            return candidate.metrics.get(metric_name, None)
    if isinstance(result, dict):
        return result.get(metric_name, None)
    if hasattr(result, "metrics"):
        return result.metrics.get(metric_name, None)
    if isinstance(result, str):
        match = re.search(r'score:\s*([\d.]+)', result)
        if match:
            try:
                return float(match.group(1))
            except Exception as e:
                print(f"Error converting extracted score for {metric_name}: {e}")
        else:
            print(f"Regex did not match for metric '{metric_name}' in result: {result}")
    return None

def evaluate_predictions(questions, references, predictions, model_name, batch_size=1, debug=True, default_score=0.5):
    test_cases = [
        LLMTestCase(input=q, actual_output=pred, expected_output=ref, context=[ref])
        for q, ref, pred in zip(questions, references, predictions)
    ]

    results = {}
    for i in tqdm(range(0, len(test_cases), batch_size), desc=f"Evaluating {model_name}"):
        batch = test_cases[i:i+batch_size]
        try:
            eval_results = evaluate(test_cases=batch, metrics=metrics, skip_on_missing_params=True)
        except KeyError as e:
            print(f"Warning: KeyError encountered for batch {i}: {e}")
            # Assign default scores for each test case in this batch.
            for _ in batch:
                for metric in metrics:
                    metric_name = get_metric_name(metric)
                    results[metric_name] = results.get(metric_name, []) + [default_score]
            continue

        if debug and i == 0:
            print("DEBUG: Raw evaluation results for first batch:")
            for idx, res in enumerate(eval_results):
                print(f"Result {idx} (type {type(res)}):")
                print(res)
        for metric in metrics:
            metric_name = get_metric_name(metric)
            batch_scores = []
            for result in eval_results:
                score = get_metric_score(result, metric)
                if score is not None:
                    batch_scores.append(score)
            if batch_scores:
                results[metric_name] = results.get(metric_name, []) + batch_scores
        clear_memory()
    final_results = {name: np.mean(scores) for name, scores in results.items() if scores}
    return final_results


In [53]:
# -------------------------
# Run GEval Evaluation
# -------------------------
# Set debug=True initially; set to False once you're satisfied with the raw output.#gpt4_predictions
gemma_results = evaluate_predictions(questions, references, fine_tuned_predictions, model_name="FineTuned_Medalpaca", batch_size=1, debug=False)
# Compile results into a DataFrame.
f_results_df = pd.DataFrame({
    "Model": ["FineTuned_Medalpaca"],
    **{dim: [gemma_results.get(dim)] for dim in gemma_results}
})

print("\n### GEval Results for Gemma Medical QnA ###")
print(f_results_df.round(2))


Evaluating FineTuned_Medalpaca:   0%|          | 0/22 [00:00<?, ?it/s]

✨ You're running DeepEval's latest Factual Accuracy (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Relevance (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Coherence (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Fluency (GEval) Metric! (using gpt-3.5-turbo, strict=False, async_mode=True)...

...


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:00, ?test case/s]
Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:05,  5.91s/test case]



Metrics Summary

  - ❌ Factual Accuracy (GEval) (score: 0.277310756667762, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The text provides information about HMPV during pregnancy and gives suggestions for managing symptoms, but does not align with the expected output in terms of specific details and clarity., error: None)
  - ❌ Relevance (GEval) (score: 0.4219945271758408, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: Some information provided aligns with the topic, but lacks depth and specifics compared to the expected output., error: None)
  - ✅ Coherence (GEval) (score: 0.7825901769384657, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The message is well-organized and provides structured suggestions for managing symptoms. The information is clear and understandable., error: None)
  - ✅ Fluency (GEval) (score: 0.8216761100162291, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: Over

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

Evaluating FineTuned_Medalpaca: 100%|██████████| 22/22 [02:30<00:00,  6.83s/it]


### GEval Results for Gemma Medical QnA ###
                 Model  Factual Accuracy  Relevance  Coherence  Fluency  \
0  FineTuned_Medalpaca              0.18       0.21        0.8     0.87   

   Clinical Safety  Completeness  Empathy  Specificity  Hallucination  
0             0.23           0.2     0.98         0.24           0.82  


In [54]:
# -------------------------
# Run GEval Evaluation
# -------------------------
# Set debug=True initially; set to False once you're satisfied with the raw output.#gpt4_predictions
gpt_results = evaluate_predictions(questions, references, gpt_predictions, model_name="GPT4", batch_size=1, debug=False)

# Compile results into a DataFrame.
gpt_results_df = pd.DataFrame({
    "Model": ["GPT4"],
    **{dim: [gpt_results.get(dim)] for dim in gpt_results}
})

print("\n### GEval Results for Gemma Medical QnA ###")
print(gpt_results_df.round(2))

Evaluating GPT4:   0%|          | 0/22 [00:00<?, ?it/s]

✨ You're running DeepEval's latest Factual Accuracy (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Relevance (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Coherence (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Fluency (GEval) Metric! (using gpt-3.5-turbo, strict=False, async_mode=True)...

...


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:00, ?test case/s]
Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:06,  6.71s/test case]



Metrics Summary

  - ❌ Factual Accuracy (GEval) (score: 0.24308143131571916, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The information provided aligns with the core medical facts and addresses symptom relief and precautions during pregnancy. However, the response lacks some clarity and conciseness compared to the expected output., error: None)
  - ❌ Relevance (GEval) (score: 0.28793744994570525, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The response provides relevant information about HMPV and its impact during pregnancy, but lacks the personalized tone and specific medication recommendations found in the expected output., error: None)
  - ✅ Coherence (GEval) (score: 0.8564352218039353, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The message is clear and well-organized with structured presentation of information., error: None)
  - ✅ Fluency (GEval) (score: 0.9774555143898714, threshold: 0.7, stric

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

Evaluating GPT4: 100%|██████████| 22/22 [02:35<00:00,  7.06s/it]


### GEval Results for Gemma Medical QnA ###
  Model  Factual Accuracy  Relevance  Coherence  Fluency  Clinical Safety  \
0  GPT4              0.24       0.27       0.89     0.95             0.34   

   Completeness  Empathy  Specificity  Hallucination  
0          0.29     0.91         0.36           0.73  


In [55]:
#questions[0]

In [56]:
def load_data(file_path="/content/BioMedLM_predictions (1).csv"):
    """Load dataset and limit to a small sample for Kaggle compatibility."""
    df = pd.read_csv(file_path)
    print(f"Total examples in dataset: {len(df)}")
    gpt_predictions = df['Model_Answer'].fillna("").tolist()
    return gpt_predictions

biomed_predictions = load_data()

Total examples in dataset: 20


In [57]:
# -------------------------
# Run GEval Evaluation
# -------------------------
# Set debug=True initially; set to False once you're satisfied with the raw output.#gpt4_predictions
bio_results = evaluate_predictions(questions[:20], references[:20], biomed_predictions, model_name="Biomed", batch_size=1, debug=False)

# Compile results into a DataFrame.
bio_results_df = pd.DataFrame({
    "Model": ["Biomed"],
    **{dim: [bio_results.get(dim)] for dim in bio_results}
})

print("\n### GEval Results for Gemma Medical QnA ###")
print(bio_results_df.round(2))

Evaluating Biomed:   0%|          | 0/20 [00:00<?, ?it/s]

✨ You're running DeepEval's latest Factual Accuracy (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Relevance (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Coherence (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Fluency (GEval) Metric! (using gpt-3.5-turbo, strict=False, async_mode=True)...

...


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:00, ?test case/s]
Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:05,  5.59s/test case]



Metrics Summary

  - ❌ Factual Accuracy (GEval) (score: 0.14236812874362256, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The text does not align with the expected output provided., error: None)
  - ❌ Relevance (GEval) (score: 0.17560524781143444, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The content covers the main topic of the human papillomavirus (HPV) vaccine in relation to cervical cancer, however, there are some inaccuracies and lack of clarity in the text., error: None)
  - ❌ Coherence (GEval) (score: 0.22079207911426563, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: Minor disjointedness in the text with unclear connection between HPV, cervical cancer, and the N-terminal domain., error: None)
  - ❌ Fluency (GEval) (score: 0.2170882633014115, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The response contains multiple grammatical errors and lacks clarity., error: None)
 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

Evaluating Biomed: 100%|██████████| 20/20 [02:18<00:00,  6.94s/it]


### GEval Results for Gemma Medical QnA ###
    Model  Factual Accuracy  Relevance  Coherence  Fluency  Clinical Safety  \
0  Biomed              0.09       0.13       0.31     0.25             0.05   

   Completeness  Empathy  Specificity  Hallucination  
0          0.08     0.26         0.12            1.0  


In [58]:
def load_data(file_path="/content/PubMedGPT_predictions.csv"):
    """Load dataset and limit to a small sample for Kaggle compatibility."""
    df = pd.read_csv(file_path)
    print(f"Total examples in dataset: {len(df)}")
    gpt_predictions = df['Model_Answer'].fillna("").tolist()
    return gpt_predictions

pubmedgpt_predictions = load_data()

Total examples in dataset: 20


In [59]:
# -------------------------
# Run GEval Evaluation
# -------------------------
# Set debug=True initially; set to False once you're satisfied with the raw output.#gpt4_predictions
pubmedgpt_results = evaluate_predictions(questions[:20], references[:20], pubmedgpt_predictions, model_name="PubMedGpt", batch_size=1, debug=False)

# Compile results into a DataFrame.
pubmedgpt_results_df = pd.DataFrame({
    "Model": ["PubMedGpt"],
    **{dim: [pubmedgpt_results.get(dim)] for dim in pubmedgpt_results}
})

print("\n### GEval Results for Gemma Medical QnA ###")
print(pubmedgpt_results_df.round(2))

Evaluating PubMedGpt:   0%|          | 0/20 [00:00<?, ?it/s]

✨ You're running DeepEval's latest Factual Accuracy (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Relevance (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Coherence (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Fluency (GEval) Metric! (using gpt-3.5-turbo, strict=False, async_mode=True)...

...


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:00, ?test case/s]
Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:06,  6.53s/test case]



Metrics Summary

  - ❌ Factual Accuracy (GEval) (score: 0.12397619724251205, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The actual output is not factually accurate and contains misleading information related to the role of the N-terminal domain of the human papillomavirus (HPV) vaccine in the prevention of cervical cancer. It also includes irrelevant details about authors and their affiliations., error: None)
  - ❌ Relevance (GEval) (score: 0.18216806701216018, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The text is not on-topic and relevant to the question about HPV and cervical cancer. It is discussing eustachian tube dysfunction and related symptoms., error: None)
  - ❌ Coherence (GEval) (score: 0.19021993163923545, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: Minor disjointedness in the text with unclear flow and repetition of information., error: None)
  - ❌ Fluency (GEval) (score: 0.184226038414

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

Evaluating PubMedGpt: 100%|██████████| 20/20 [02:21<00:00,  7.06s/it]


### GEval Results for Gemma Medical QnA ###
       Model  Factual Accuracy  Relevance  Coherence  Fluency  \
0  PubMedGpt              0.09       0.12       0.35     0.23   

   Clinical Safety  Completeness  Empathy  Specificity  Hallucination  
0             0.05          0.08     0.31         0.11            1.0  


In [60]:
def load_data(file_path="/content/MedAlpaca-7B_predictions.csv"):
    """Load dataset and limit to a small sample for Kaggle compatibility."""
    df = pd.read_csv(file_path)
    print(f"Total examples in dataset: {len(df)}")
    gpt_predictions = df['Model_Answer'].fillna("").tolist()
    return gpt_predictions

openbio_predictions = load_data()

Total examples in dataset: 20


In [61]:
# -------------------------
# Run GEval Evaluation
# -------------------------
# Set debug=True initially; set to False once you're satisfied with the raw output.#gpt4_predictions
openbio_results = evaluate_predictions(questions[:20], references[:20], openbio_predictions, model_name="OpenBio", batch_size=1, debug=False)

# Compile results into a DataFrame.
openbio_results_df = pd.DataFrame({
    "Model": ["OpenBio"],
    **{dim: [openbio_results.get(dim)] for dim in openbio_results}
})

print("\n### GEval Results for Gemma Medical QnA ###")
print(openbio_results_df.round(2))

Evaluating OpenBio:   0%|          | 0/20 [00:00<?, ?it/s]

✨ You're running DeepEval's latest Factual Accuracy (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Relevance (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Coherence (GEval) Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Fluency (GEval) Metric! (using gpt-3.5-turbo, strict=False, async_mode=True)...

...


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:00, ?test case/s]
Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:04,  4.24s/test case]



Metrics Summary

  - ❌ Factual Accuracy (GEval) (score: 0.17304032759579033, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The information provided does not align with the expected output, as it mentions middle ear infection and outer ear canal inflammation instead of eustachian tube dysfunction. Clinical evaluation and investigation recommendations are similar., error: None)
  - ❌ Relevance (GEval) (score: 0.21459758820697572, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: Significant differences in the details provided between Actual Output and Expected Output., error: None)
  - ✅ Coherence (GEval) (score: 0.7837568646074996, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The text is well-organized and presents information about the diagnosis and treatment of Earache, but slightly lacks structured presentation of ideas., error: None)
  - ❌ Fluency (GEval) (score: 0.5368355865986054, threshold: 0.7, strict: 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

Evaluating OpenBio: 100%|██████████| 20/20 [01:59<00:00,  5.98s/it]


### GEval Results for Gemma Medical QnA ###
     Model  Factual Accuracy  Relevance  Coherence  Fluency  Clinical Safety  \
0  OpenBio              0.16       0.18       0.56     0.42             0.16   

   Completeness  Empathy  Specificity  Hallucination  
0          0.17     0.56         0.22           0.85  


In [62]:
import pandas as pd
import plotly.express as px


# Combine both DataFrames
combined_df = pd.concat([f_results_df, gpt_results_df,openbio_results_df,bio_results_df,pubmedgpt_results_df])

# Reshape to long format
df_long = combined_df.melt(id_vars="Model", var_name="Metric", value_name="Score")

# Plotly bar chart
fig = px.bar(
    df_long,
    x="Metric",
    y="Score",
    color="Model",
    barmode="group",
    title="📊 Model Comparison on GEval Metrics",
    template="plotly_white"
)

fig.update_layout(
    xaxis_title="Evaluation Metric",
    yaxis_title="Score",
    legend_title="Model",
    height=500,
    width=900
)

fig.show()
